# Exploring MAUDE Database

The data used in this analysis is from the Manufacturer and User Facility Device Experience (MAUDE) Database provided through the openFDA (https://open.fda.gov/).

The openFDA has datasets covering:
1. Pharmaceuticals
2. Medical Devices
3. Food

The datasets cover a large variety of information, such as:
- Adverse Events
- Labeling
- Registrations 
- Recalls
- Classification 

## MAUDE 

The purpose of this project is to dive into the Federal Food and Drug Administration's (FDA) Medical Device Adverse Event Database (MAUDE).  Medical devices range from band-aids to artificial hearts and x-ray imagers, and are regulated using a risk based approach.  An adverse event is any untoward medical condition that is associated with the medical device, or may be associated with the medical device. 

Manufacturer's a required to maintain and evaluated complaint, including potential adverse events, for their devices. In addition, MAUDE was developed by the FDA as an additional platform for reporting adverse events.  There is a limited amount of control in how thoroughly the adverse event is reported.  For that reason, there is a lot of missing data in this dataset.

The initial json request sent, for all medical device advese events between January 1, 2017 - December 31, 2017 returned a total count of 935708 adverse events.  For this reason, a subset of the data will be used for the initial analysis.  

In [79]:
import pandas as pd 
import numpy as np 
import urllib.request, json
from pandas.io.json import json_normalize
import datetime

# Examining the Data

The initial form of the data is a JSON structured nested dictionary, which will require multiple iterations to flatten and maintain the desired information.  This will be done initially using the json_normalize function.  The beginning structure of the adverse event is:

<img src="openfda_json.png">

All of the information of interest is housed in "results."

## First Normalization

In [80]:
# Returning 100 adverse events in a json dictionary from the openFDA API  
with urllib.request.urlopen("https://api.fda.gov/device/event.json?search=date_received:[20170101+TO+20171231]&limit=100") as url:
    data = json.loads(url.read())

# Normalize the data based on the "results" section of the JSON
# First normalization to see the structure of the data
maude_general = json_normalize(data['results'])

maude_general.head()

,adverse_event_flag,date_facility_aware,date_manufacturer_received,date_of_event,date_received,date_report,device,device_date_of_manufacturer,distributor_address_1,distributor_address_2,...,removal_correction_number,report_number,report_source_code,report_to_fda,report_to_manufacturer,reporter_occupation_code,reprocessed_and_reused_flag,single_use_flag,source_type,type_of_report
0,Y,NaN,20110503,20100101,20170112,20110421,[],NaN,,,...,,1226230-2011-00002,Manufacturer report,N,,PHYSICIAN,I,Y,[HEALTH PROFESSIONAL],"[Followup, Followup, Followup, Followup, Follo..."
1,N,NaN,20150810,NaN,20170309,NaN,"[{'manufacturer_d_address_1': '', 'manufacture...",20030314,,,...,,2210968-2015-02964,Manufacturer report,N,,,I,*,[],[Followup]
2,N,NaN,20170802,20161205,20170101,20161229,[{'manufacturer_d_address_1': '900 WALNUT RIDG...,20150721,,,...,,2183926-2016-00829,Manufacturer report,N,,OTHER,N,N,[USER FACILITY],[Initial submission]
3,N,NaN,20170221,20161209,20170101,20161209,[{'manufacturer_d_address_1': '900 WALNUT RIDG...,20150721,,,...,,2183926-2016-00828,Manufacturer report,N,,OTHER,N,N,[USER FACILITY],[Initial submission]
4,N,NaN,20170928,20161208,20170101,20161208,[{'manufacturer_d_address_1': '900 WALNUT RIDG...,20150330,,,...,,2183926-2016-00830,Manufacturer report,N,,OTHER,N,N,[USER FACILITY],[Initial submission]


## Second Normalization

The second stage of normalization was done focusing on the nested dictionary "device", where the majority of the desired information is housed.  Additional information from "results" is appended to the flattened table as well for each adverse event. A lot of the columns from results were ignored, since previewing the data for another project demonstrated that those columns are empty >90% of the time.

In [81]:
# Most of the imformation is housed within the "device" section of the "results
maude_device = json_normalize(data=data['results'], record_path='device', 
        # Appending meta data to the end of each row with information we want to keep from "results"
        meta=['adverse_event_flag', 'date_manufacturer_received','date_of_event', 
        'event_key', 'event_location', 'event_type',
        'health_professional', 'manufacturer_city', 'manufacturer_country'],
        errors='ignore')

In [82]:
maude_device.head()

,brand_name,catalog_number,date_received,date_removed_flag,date_returned_to_manufacturer,device_age_text,device_availability,device_evaluated_by_manufacturer,device_event_key,device_operator,...,other_id_number,adverse_event_flag,date_manufacturer_received,date_of_event,event_key,event_location,event_type,health_professional,manufacturer_city,manufacturer_country
0,MESH,,20170309,,NaN,DA,Yes,N,,HEALTH PROFESSIONAL,...,,N,20150810,nan,,I,Injury,,,
1,MERGE HEMODYNAMICS,,20170101,,NaN,DA,Yes,R,,HEALTH PROFESSIONAL,...,,N,20170802,20161205,,I,Malfunction,N,,
2,MERGE HEMODYNAMICS,,20170101,,20170220,DA,Device was returned to manufacturer,Y,,HEALTH PROFESSIONAL,...,,N,20170221,20161209,,I,Malfunction,N,,
3,MERGE UNITY PACS,,20170101,,NaN,DA,Yes,Y,,HEALTH PROFESSIONAL,...,,N,20170928,20161208,,I,Malfunction,N,,
4,OT VERIO FLEX METER,,20170101,,NaN,,No,R,,LAY USER/PATIENT,...,1-3BBSA4F,N,20161224,nan,,I,Malfunction,N,,


## Final Flattening Step

Good information is housed within the "device" section, in a sub-dictionary called "openfda."  This contains the device classification, generic medical name and the medical specialty.

This was extracted using a function that takes the information from the dictionary and returns it in a list.  Then was applied to the dataframe, converted from a series of lists to a dataframe, and transposed.  Then it was added to the main dataset after conforming they are equal lengths.  

In [83]:
# A function to extract the desired data from the openfda column
def get_openfda(open_dict):
    try:
        # The data is housed in a json dictionary structure
        device_name = open_dict['device_name']
        device_class = open_dict['device_class']
        medical_specialty = open_dict['medical_specialty_description']
    except:
        return [np.NaN, np.NaN, np.NaN]
    # returning all of the captured information
    return [device_name, device_class, medical_specialty]

In [84]:
# Creating a series containing all of the results using df.apply
openfda = maude_device['openfda'].apply(get_openfda)

# Converting the lists in the series to a dataframe
openfda_df = pd.DataFrame.from_items(zip(openfda.index, openfda.values))
openfda_df = openfda_df.transpose()
openfda_df.columns = ['general_device_name', 'device_class', 'medical_specialty']

In [85]:
# Making sure the maude_device dataframe and the openfda_df dataframe are the same length before merging
print("The length of the dataframes are: {}".format([len(maude_device), len(openfda_df)]))

openfda_df.head()

The length of the dataframes are: [99, 99]


,general_device_name,device_class,medical_specialty
0,"Mesh, Surgical, Polymeric",2,"General, Plastic Surgery"
1,"Computer, Diagnostic, Programmable",2,Cardiovascular
2,"Computer, Diagnostic, Programmable",2,Cardiovascular
3,"System, Image Processing, Radiological",2,Radiology
4,"System, Test, Blood Glucose, Over The Counter",2,Clinical Chemistry


In [86]:
# Combinding the two dataframes
maude_tot = pd.concat([maude_device, openfda_df], axis=1)
# Removing the initial openfda column
maude_tot.drop('openfda', axis=1, inplace=True)
maude_tot.head()

,brand_name,catalog_number,date_received,date_removed_flag,date_returned_to_manufacturer,device_age_text,device_availability,device_evaluated_by_manufacturer,device_event_key,device_operator,...,date_of_event,event_key,event_location,event_type,health_professional,manufacturer_city,manufacturer_country,general_device_name,device_class,medical_specialty
0,MESH,,20170309,,NaN,DA,Yes,N,,HEALTH PROFESSIONAL,...,nan,,I,Injury,,,,"Mesh, Surgical, Polymeric",2,"General, Plastic Surgery"
1,MERGE HEMODYNAMICS,,20170101,,NaN,DA,Yes,R,,HEALTH PROFESSIONAL,...,20161205,,I,Malfunction,N,,,"Computer, Diagnostic, Programmable",2,Cardiovascular
2,MERGE HEMODYNAMICS,,20170101,,20170220,DA,Device was returned to manufacturer,Y,,HEALTH PROFESSIONAL,...,20161209,,I,Malfunction,N,,,"Computer, Diagnostic, Programmable",2,Cardiovascular
3,MERGE UNITY PACS,,20170101,,NaN,DA,Yes,Y,,HEALTH PROFESSIONAL,...,20161208,,I,Malfunction,N,,,"System, Image Processing, Radiological",2,Radiology
4,OT VERIO FLEX METER,,20170101,,NaN,,No,R,,LAY USER/PATIENT,...,nan,,I,Malfunction,N,,,"System, Test, Blood Glucose, Over The Counter",2,Clinical Chemistry


# Expanding the Dataset

Now that the data is flattened appropriately, the same methods for flattening can be applied to a larger dataset.  To avoid the slow down from sending a lot of incremented queries, since the openFDA API only returns a maximum of 100 items for each request, the data is also provided in JSON files.  


In [87]:
def get_json(json_file):
    with open(json_file) as f:
        data = json.load(f)
    return data

def normalize_maude(data):
    # Normalize the data based on the "results" section of the JSON
    # First normalization to see the structure of the data
    maude_general = json_normalize(data['results'])
    
    # Most of the imformation is housed within the "device" section of the "results
    maude_device = json_normalize(data=data['results'], record_path='device', 
    # Appending meta data to the end of each row with information we want to keep from "results"
         meta=['adverse_event_flag', 'date_manufacturer_received','date_of_event', 
         'event_key', 'event_location', 'event_type',
         'health_professional', 'manufacturer_city', 'manufacturer_country'],
        errors='ignore')
    
    # Creating a series containing all of the results using df.apply
    openfda = maude_device['openfda'].apply(get_openfda)
    
    # Converting the lists in the series to a dataframe
    openfda_df = pd.DataFrame.from_items(zip(openfda.index, openfda.values))
    openfda_df = openfda_df.transpose()
    openfda_df.columns = ['general_device_name', 'device_class', 'medical_specialty']
    
    # Combinding the two dataframes
    maude_tot = pd.concat([maude_device, openfda_df], axis=1)
    # Removing the initial openfda column
    maude_tot.drop('openfda', axis=1, inplace=True)
    return maude_tot


In [88]:
# Importing part of the dataset for Q1 2017
data_2017 = get_json('Datasets/device-event-0001-of-0003.json')

In [103]:
# Flattening all of the datasets
flat_2017 = normalize_maude(data_2017)
flat_2017.head()

,brand_name,catalog_number,date_received,date_removed_flag,date_returned_to_manufacturer,device_age_text,device_availability,device_evaluated_by_manufacturer,device_event_key,device_operator,...,date_of_event,event_key,event_location,event_type,health_professional,manufacturer_city,manufacturer_country,general_device_name,device_class,medical_specialty
0,"LIBERTY CYCLER SET, SINGLE CONN./EXT. DL",050-87216,20170125,,NaN,MO,No,R,,LAY USER/PATIENT,...,20161229,,I,Malfunction,N,,,"System, Peritoneal, Automatic Delivery",2,"Gastroenterology, Urology"
1,FINESSIS ZERO CHEMO FLEXYLON POWDER FREE,,20170308,,NaN,,No,I,,HEALTH PROFESSIONAL,...,20170301,,NO INFORMATION,Injury,*,,,Surgeon'S Gloves,1,"General, Plastic Surgery"
2,"DIASTAT ANTINUCLEAR ANTIBODY, ANTIGEN, CONTROL",FANA200,20170306,,NaN,,No,R,,BIOMEDICAL ENGINEER,...,20170130,,I,Malfunction,Y,,,"Antinuclear Antibody, Antigen, Control",2,Immunology
3,SPINE TRACKER,6002-005-000,20170222,,NaN,DA,No,R,,HEALTH PROFESSIONAL,...,20170126,,I,Malfunction,N,,,Neurological Stereotaxic Instrument,2,Neurology
4,ROCHESTER MAGIC3 COUDE HYDROPHLIC 16FR MALE 30BX,50616,20170213,,20170201,DA,Device was returned to manufacturer,Y,,LAY USER/PATIENT,...,nan,,I,Malfunction,N,,,"Catheter, Urological",2,"Gastroenterology, Urology"


## Exploring the data

In [104]:
print("The number of adverse events in the dataset: {}".format(flat_2017.shape[0]))

flat_2017.info()

The number of adverse events in the dataset: 100629
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100629 entries, 0 to 100628
Data columns (total 39 columns):
brand_name                          100629 non-null object
catalog_number                      100629 non-null object
date_received                       100629 non-null object
date_removed_flag                   100629 non-null object
date_returned_to_manufacturer       37193 non-null object
device_age_text                     100629 non-null object
device_availability                 100629 non-null object
device_evaluated_by_manufacturer    100629 non-null object
device_event_key                    100629 non-null object
device_operator                     100629 non-null object
device_report_product_code          100629 non-null object
device_sequence_number              100629 non-null object
expiration_date_of_device           19654 non-null object
generic_name                        100629 non-null object
implant_flag 

It appears there's not a lot of null values in our dataset, but on visual inspection there is clearly a lot of blank columns.  To get a better idea of how much data is missing, the missing data will be replaced with null values.  

## Data Cleaning

In [111]:
# Replacing missing data with null values
flat_2017 = flat_2017.replace({'': np.NaN, 'nan': np.NaN, 'NaN': np.NaN})
# Looking at the number of null values in each column
flat_null = flat_2017.isnull().sum()
flat_null

brand_name                             709
catalog_number                       29008
date_received                            0
date_removed_flag                   100629
date_returned_to_manufacturer        63436
device_age_text                      61142
device_availability                      0
device_evaluated_by_manufacturer         1
device_event_key                    100629
device_operator                          0
device_report_product_code               0
device_sequence_number                   0
expiration_date_of_device            80975
generic_name                           517
implant_flag                        100629
lot_number                           53071
manufacturer_d_address_1              5365
manufacturer_d_address_2             79859
manufacturer_d_city                   2321
manufacturer_d_country                6423
manufacturer_d_name                    339
manufacturer_d_postal_code            8775
manufacturer_d_state                 18694
manufacture

In [112]:
# Filtering out columns that are missing more than 30% of the data
null_33 = flat_null[flat_null<(len(flat_2017)/3)].index
flat_less_null = flat_2017[null_33].copy()
flat_less_null.head()

,brand_name,catalog_number,date_received,device_availability,device_evaluated_by_manufacturer,device_operator,device_report_product_code,device_sequence_number,generic_name,manufacturer_d_address_1,...,model_number,adverse_event_flag,date_manufacturer_received,date_of_event,event_location,event_type,health_professional,general_device_name,device_class,medical_specialty
0,"LIBERTY CYCLER SET, SINGLE CONN./EXT. DL",050-87216,20170125,No,R,LAY USER/PATIENT,FKX,1.0,"SYSTEM, PERITONEAL, AUTOMATED DELIVERY",MIKE ALLEN #1331,...,NaN,N,20170126,20161229,I,Malfunction,N,"System, Peritoneal, Automatic Delivery",2,"Gastroenterology, Urology"
1,FINESSIS ZERO CHEMO FLEXYLON POWDER FREE,NaN,20170308,No,I,HEALTH PROFESSIONAL,KGO,1.0,FLEXYLON SURGICAL GLOVE,NaN,...,FINESSIS ZERO CHEMO FLEXYLON,N,NaN,20170301,NO INFORMATION,Injury,*,Surgeon'S Gloves,1,"General, Plastic Surgery"
2,"DIASTAT ANTINUCLEAR ANTIBODY, ANTIGEN, CONTROL",FANA200,20170306,No,R,BIOMEDICAL ENGINEER,LKJ,1.0,ANA,P. O. BOX 50117,...,NaN,N,20170202,20170130,I,Malfunction,Y,"Antinuclear Antibody, Antigen, Control",2,Immunology
3,SPINE TRACKER,6002-005-000,20170222,No,R,HEALTH PROFESSIONAL,HAW,1.0,NEUROLOGICAL STEREOTAXIC INSTRUMENTS,4100 EAST MILHAM AVENUE,...,6002-005-000,N,20170126,20170126,I,Malfunction,N,Neurological Stereotaxic Instrument,2,Neurology
4,ROCHESTER MAGIC3 COUDE HYDROPHLIC 16FR MALE 30BX,50616,20170213,Device was returned to manufacturer,Y,LAY USER/PATIENT,KOD,1.0,MAGIC 3,8195 INDUSTRIAL BLVD,...,NaN,N,20180118,NaN,I,Malfunction,N,"Catheter, Urological",2,"Gastroenterology, Urology"


Prior to dropping the rows containing NaN values, a few more columns that probably don't hold useful information for analysis will be dropped.  In addition repeatative columns, such as multiple instances of the manufacturer's location will be dropped or postal code and zip code. 

In [113]:
# Dropping redundant or non relevant columns
not_relevant = ['catalog_number', 'model_number', 'manufacturer_d_address_1','manufacturer_d_postal_code']

flat_less_null.drop(not_relevant, inplace=True, axis=1)
flat_less_null.head()

,brand_name,date_received,device_availability,device_evaluated_by_manufacturer,device_operator,device_report_product_code,device_sequence_number,generic_name,manufacturer_d_city,manufacturer_d_country,...,manufacturer_d_zip_code,adverse_event_flag,date_manufacturer_received,date_of_event,event_location,event_type,health_professional,general_device_name,device_class,medical_specialty
0,"LIBERTY CYCLER SET, SINGLE CONN./EXT. DL",20170125,No,R,LAY USER/PATIENT,FKX,1.0,"SYSTEM, PERITONEAL, AUTOMATED DELIVERY",REYNOSA,MX,...,88780,N,20170126,20161229,I,Malfunction,N,"System, Peritoneal, Automatic Delivery",2,"Gastroenterology, Urology"
1,FINESSIS ZERO CHEMO FLEXYLON POWDER FREE,20170308,No,I,HEALTH PROFESSIONAL,KGO,1.0,FLEXYLON SURGICAL GLOVE,BAAR,NaN,...,CH634,N,NaN,20170301,NO INFORMATION,Injury,*,Surgeon'S Gloves,1,"General, Plastic Surgery"
2,"DIASTAT ANTINUCLEAR ANTIBODY, ANTIGEN, CONTROL",20170306,No,R,BIOMEDICAL ENGINEER,LKJ,1.0,ANA,"MALMO, SKANE",SW,...,20211,N,20170202,20170130,I,Malfunction,Y,"Antinuclear Antibody, Antigen, Control",2,Immunology
3,SPINE TRACKER,20170222,No,R,HEALTH PROFESSIONAL,HAW,1.0,NEUROLOGICAL STEREOTAXIC INSTRUMENTS,KALAMAZOO,US,...,49001,N,20170126,20170126,I,Malfunction,N,Neurological Stereotaxic Instrument,2,Neurology
4,ROCHESTER MAGIC3 COUDE HYDROPHLIC 16FR MALE 30BX,20170213,Device was returned to manufacturer,Y,LAY USER/PATIENT,KOD,1.0,MAGIC 3,COVINGTON,US,...,30014,N,20180118,NaN,I,Malfunction,N,"Catheter, Urological",2,"Gastroenterology, Urology"


In [126]:
maude_final = flat_less_null.dropna().copy()
print("Remaining datapoints: {}".format(maude_final.shape[0]))

Remaining datapoints: 62629


The next step will be to convert all of the dates to datetime objects.

In [128]:
# Selecting all columns that contain date
date_columns = maude_final.columns[maude_final.columns.str.contains('date')]

for col in date_columns:
    maude_final[col] = pd.to_datetime(maude_final[col], errors='coerce')
    
maude_final[date_columns].head()

,date_received,date_manufacturer_received,date_of_event
3,2017-02-22,2017-01-26,2017-01-26
5,2017-01-14,2016-12-27,2016-12-28
11,2017-02-15,2017-01-19,2016-11-12
14,2017-01-04,2016-12-06,2016-12-06
16,2017-01-23,2017-01-03,2016-11-29
